In [ ]:
!pip install delta-spark==2.4.0

In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from delta.tables import *
from pyspark.sql.functions import *

In [ ]:
spark_conf = SparkConf()
spark_conf.set("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0,com.amazonaws:aws-java-sdk-bundle:1.11.1026,org.apache.hadoop:hadoop-aws:3.3.4")
spark_conf.set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
spark_conf.set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
spark_conf.set("spark.cores", "8")
spark_conf.set("spark.driver.cores", "8")
spark_conf.set("hive.metastore.uris", "thrift://hive-metastore:9083")

spark = SparkSession \
    .builder.master("local") \
    .appName("Example") \
    .config(conf=spark_conf) \
    .enableHiveSupport() \
    .getOrCreate()

In [ ]:
spark.sql(f"""
    CREATE DATABASE IF NOT EXISTS datalake
    LOCATION 's3a://raw.datalake.mydomain.com/';
""").show()

In [ ]:
data = spark.range(0, 1000000)

In [ ]:
data.show(5)

In [ ]:
data.write.format("delta").mode("overwrite").saveAsTable("datalake.simple_table")

In [ ]:
spark.sql("show tables from datalake").show()

In [ ]:
spark.sql("select count(1) from datalake.simple_table").show()

# Delta Lake table update

In [ ]:
spark.sql(f"""DELETE FROM datalake.simple_table WHERE (id % 2) = 0""").show()

In [ ]:
data = spark.range(0, 50000000)

In [ ]:
data.write.format("delta").mode("append").saveAsTable("datalake.simple_table")

# Delta Table Vacuum

In [ ]:
table = DeltaTable.forPath(spark, "s3a://raw.datalake.mydomain.com/simple_table")

In [ ]:
table.vacuum()